## Training

In [ ]:
import boto3
import sagemaker

print(sagemaker.__version__)

session = sagemaker.Session()
bucket = session.default_bucket()

In [ ]:
prefix = 'amazon-reviews'

# Update s3_train_path and s3_val_path to the outputs of the SageMaker Processing job
s3_train_path = 's3://sagemaker-eu-west-1-613904931467/sagemaker-scikit-learn-2021-06-18-09-07-03-998/output/train_data'
s3_val_path = 's3://sagemaker-eu-west-1-613904931467/sagemaker-scikit-learn-2021-06-18-09-07-03-998/output/validation_data'

s3_output = 's3://{}/{}/output/'.format(bucket, prefix)

print(s3_train_path)
print(s3_val_path)
print(s3_output)

In [ ]:
from sagemaker import image_uris

region = boto3.Session().region_name    
container = image_uris.retrieve('blazingtext', region)
print(container)

In [ ]:
role = sagemaker.get_execution_role()

#instance_type = 'ml.c5.2xlarge'
#instance_type = 'ml.c5.4xlarge'
instance_type = 'ml.c5.9xlarge'
#instance_type = 'ml.c5.18xlarge'
#instance_type = 'ml.p3.2xlarge'

bt = sagemaker.estimator.Estimator(container,
                                   role, 
                                   instance_count=1, 
                                   instance_type=instance_type,
                                   output_path=s3_output)

In [ ]:
bt.set_hyperparameters(mode='supervised',
                       vector_dim=300, 
                       word_ngrams=3, 
                       epochs=50)

In [ ]:
train_data = sagemaker.TrainingInput(s3_train_path, content_type='text/plain')
validation_data = sagemaker.TrainingInput(s3_val_path,content_type='text/plain')

s3_channels = {'train': train_data, 'validation': validation_data}

In [ ]:
bt.fit(inputs=s3_channels)

In [ ]:
bt_predictor = bt.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

In [ ]:
import json
import pprint

sentences = ['This is a bad camera it doesnt work at all , i want a refund  .', 
             'The camera works , the pictures are decent quality, nothing special to say about it .',
             'Very happy to have bought this , exactly what I needed']

payload = {"instances" : sentences, "configuration": {"k": 3}}

bt_predictor.serializer = sagemaker.serializers.JSONSerializer()
response = bt_predictor.predict(payload)

print(response)

In [ ]:
bt_predictor.delete_endpoint()